In [1]:
import math
from liblibra_core import *
from libra_py import build
from libra_py import units
from libra_py import autoconnect

import py3Dmol

import matplotlib.pyplot as plt
#from matplotlib.pyplot import figure
%matplotlib inline

plt.rc('axes', titlesize=24)      # fontsize of the axes title
plt.rc('axes', labelsize=20)      # fontsize of the x and y labels
plt.rc('legend', fontsize=20)     # legend fontsize
plt.rc('xtick', labelsize=16)    # fontsize of the tick labels
plt.rc('ytick', labelsize=16)    # fontsize of the tick labels

plt.rc('figure.subplot', left=0.2)
plt.rc('figure.subplot', right=0.95)
plt.rc('figure.subplot', bottom=0.13)
plt.rc('figure.subplot', top=0.88)

colors = {}
colors.update({"11": "#8b1a0e"})  # red       
colors.update({"12": "#FF4500"})  # orangered 
colors.update({"13": "#B22222"})  # firebrick 
colors.update({"14": "#DC143C"})  # crimson   

colors.update({"21": "#5e9c36"})  # green
colors.update({"22": "#006400"})  # darkgreen  
colors.update({"23": "#228B22"})  # forestgreen
colors.update({"24": "#808000"})  # olive      

colors.update({"31": "#8A2BE2"})  # blueviolet
colors.update({"32": "#00008B"})  # darkblue  

colors.update({"41": "#2F4F4F"})  # darkslategray

/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: to-Python converter for std::vector<std::complex<double>, std::allocator<std::complex<double> > > already registered; second conversion method ignored.
  
/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  
/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator

In this tutorial, we are going to demonstrate some of the capabilities of the **build** and **autoconnect** modules. The content (in terms of functions available) of these modules is summarized below.

#### build

* **read_xyz(filename)**
* **make_xyz(L, R)**
* read_xyz_crystal(filename, a,b,c)
* **generate_replicas_xyz2(L, R, tv1, tv2, tv3, Nx, Ny, Nz)**
* generate_replicas_xyz(tv1,tv2,tv3, rep1, rep2, rep3 , filename, outfile)
* crop_sphere_xyz2(L, R, Rcut)
* **crop_sphere_xyz3(L, R, Rcut, pairs, new_L)**
* crop_sphere_xyz(infile, outfile, Rcut)
* add_atoms_to_system(syst, L, R, scl, mass, univ_file)
* **add_atom_to_system(syst, coords, MaxCoords, Nx,Ny,Nz, a,b,c, shift, elt, mass, scl, max_coord, rnd)**


#### autoconnect
* autoconnect(R, MaxCoord, Rcut, opt=0, verbosity=0)
* **autoconnect_pbc(R, MaxCoord, Rcut, tv1, tv2, tv3, pbc_opt, opt=0, verbosity=0)**
* example_1()

The functions used in this tutorial are shown in bold.

Define a unit cell parameters - periodic translations of the minimal unit cell (provided as an input file)

In [2]:
a = VECTOR(5.833, 0.0, 0.0) * units.Angst
b = VECTOR(0.0, 5.833, 0.0) * units.Angst
c = VECTOR(0.0, 0.0, 5.833) * units.Angst

Read the minimal unit cell information from the file

In [3]:
L0, R0 = build.read_xyz("CdSe.xyz")
print L0, R0

['Cd', 'Cd', 'Cd', 'Cd', 'Se', 'Se', 'Se', 'Se'] [<liblibra_core.VECTOR object at 0x7f30f3259490>, <liblibra_core.VECTOR object at 0x7f30f2ed15a0>, <liblibra_core.VECTOR object at 0x7f30f2ed17c0>, <liblibra_core.VECTOR object at 0x7f30f2ed1848>, <liblibra_core.VECTOR object at 0x7f30f2ed18d0>, <liblibra_core.VECTOR object at 0x7f30f2ed1958>, <liblibra_core.VECTOR object at 0x7f30f2ed19e0>, <liblibra_core.VECTOR object at 0x7f30f2ed1a68>]


In [4]:
xyz = build.make_xyz(L0, R0)
view = py3Dmol.view(width=800,height=400)  # linked=False,viewergrid=(3,2)
view.setBackgroundColor('0xeeeeee')
view.zoomTo()
view.addModel(xyz,'xyz',{'vibrate': {'frames':10,'amplitude':1.0}})
view.setStyle({'sphere':{'colorscheme':'Jmol'}})
view.show()

print xyz

You appear to be running in JupyterLab. You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

8

Cd     0.00000000    0.00000000    0.00000000 
Cd     2.91650000    2.91650000    0.00000000 
Cd     2.91650000    0.00000000    2.91650000 
Cd     0.00000000    2.91650000    2.91650000 
Se     1.45825000    1.45825000    1.45825000 
Se     1.45825000    4.37475000    4.37475000 
Se     4.37475000    1.45825000    4.37475000 
Se     4.37475000    4.37475000    1.45825000 



Now, lets create a super-cell by replicating the minimal cell in 3 directions
We need large enough super-cell so that the QDs we are going to create fits inside it
so, replicate 6 times in each direction
This will generate the labels and coordinates of the new super-system

In [5]:
Nx, Ny, Nz = 6, 6, 6 
L1, R1 = build.generate_replicas_xyz2(L0, R0, a, b, c, Nx, Ny, Nz)

print L1, len(R1)

xyz = build.make_xyz(L1, R1)
view = py3Dmol.view(width=800,height=400)  # linked=False,viewergrid=(3,2)
view.setBackgroundColor('0xeeeeee')
view.zoomTo()
view.addModel(xyz,'xyz',{'vibrate': {'frames':10,'amplitude':1.0}})
view.setStyle({'sphere':{'colorscheme':'Jmol'}})
view.show()

['Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd', 'Cd

You appear to be running in JupyterLab. You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Next, we'll be connecting atoms in the super-cell to each other, based on their separation distances
and the maximal coordination numbers accepted
At this point, we are going to define the maximal coordination number of each atom type present in the 
system (here, Cd and Se), as well as for each additional elements - e.g. those which we are going to cap
the QD with (here, H)

In [6]:
PT_coord = { "Cd": 4, "Se": 4, "H":1 }
MaxCoord1 = []
masses = []
for i in L1:
    MaxCoord1.append(PT_coord[i])    # maximal coordination number of each atom in super-cell
    masses.append(1.0)               # masses of each atom in the super-cell, do not matter here

The parameter that defines the atomic connectivities: if the atoms are separated
the the distance shorter than this, and if their actual coordination number has not
rached the maximal one yet, we will assume they are covelently bonded (in FF there will
be a harmonic potential term for instance)

In [7]:
Rbond = 3.001 * units.Angst

Now connect all atoms in the supercell. 
The "none" indicates that there is no periodic boundary conditions to check when doing such connections

In [8]:
res, line, pairs = autoconnect.autoconnect_pbc(R1, MaxCoord1, Rbond, Nx*a, Ny*b, Nz*c, "none", 0, 0)

The radius of the QD 

In [9]:
Rdot = 7.0 * units.Angst

Finally, we can carve a QD out of the super-cell. All the atoms that are outside the shepre of Rdot radius
are removed, except for the atoms that are bound (as encoded in the "pairs" argument) to the atoms inside
the sphere. The bound outside atoms' identities will be switched according to which atoms there are connected.
This is encoded in the last (dictionary) argument. Here, it means that all the atoms outside the sphere that
are connected to the Cd inside will become H, all the outside atoms that are connected to the inside Se will
be converted to Cd.In other words, all Se atoms will be capped by Cd and all Cd atoms will be capped with H.
The L and R variables will contain the labels and coordinates of the final (QD) atoms.

In [10]:
L, R = build.crop_sphere_xyz3(L1, R1, Rdot, pairs, {"Cd":"H", "Se":"Cd"})

xyz = build.make_xyz(L, R)
view = py3Dmol.view(width=800,height=400)  # linked=False,viewergrid=(3,2)
view.setBackgroundColor('0xeeeeee')
view.zoomTo()
view.addModel(xyz,'xyz',{'vibrate': {'frames':10,'amplitude':1.0}})
view.setStyle({'sphere':{'colorscheme':'Jmol'}})
view.show()

You appear to be running in JupyterLab. You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [13]:
syst1 = System()
rnd = Random()

# Unit cell parameters
a = VECTOR(5.0, 5.0, 0.0) * units.Angst
b = VECTOR(0.0, 5.0, 0.0) * units.Angst
c = VECTOR(0.0, 0.0, 5.0) * units.Angst

# Create a Chemical system
Nx, Ny, Nz = 5, 5, 5  # 

R, MaxCoord = [], []
build.add_atom_to_system(syst1, R, MaxCoord, Nx,Ny,Nz, a, b, c, VECTOR(0.0, 0.0, 0.0) * units.Angst, "Na", 2000.0, VECTOR(1.0, 1.0, 1.0), 6, rnd)
build.add_atom_to_system(syst1, R, MaxCoord, Nx,Ny,Nz, a, b, c, VECTOR(2.5, 2.5, 2.5) * units.Angst, "Cl", 2000.0, VECTOR(1.0, 1.0, 1.0), 6, rnd)

In [14]:
xyz = syst1.get_xyz(0, "abc", 0)
view = py3Dmol.view(width=800,height=400)  # linked=False,viewergrid=(3,2)
view.setBackgroundColor('0xeeeeee')
view.zoomTo()
view.addModel(xyz,'xyz',{'vibrate': {'frames':10,'amplitude':1.0}})
view.setStyle({'sphere':{'colorscheme':'Jmol'}})
view.show()

You appear to be running in JupyterLab. You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol